In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import dotenv

dotenv.load_dotenv()

llama8b = "meta-llama/Meta-Llama-3-8B-Instruct"

True

In [18]:
def insert_between_tokens(model_id, prompt, max_length, insert_str):
    model = AutoModelForCausalLM.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    generated = input_ids
    
    insert_toks = tokenizer.encode(insert_str, return_tensors='pt', add_special_tokens=False, )
    print(insert_toks.shape)
    
    while generated.shape[1] < max_length:
        model_inputs = model.prepare_inputs_for_generation(generated)
        with torch.no_grad():
            outputs = model(**model_inputs)
        next_token = outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(0)
        generated = torch.cat([generated, next_token], dim=-1)
        generated = torch.cat([generated, insert_toks], dim=-1)
    
    return tokenizer.decode(generated[0])


prompt = "Alternate saying '1' and '0'. 0101010"
insert_str = '101'
result = insert_between_tokens(llama8b, prompt, max_length=100, insert_str=insert_str)
print(result)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.96it/s]


<|begin_of_text|>Alternate saying '1' and '0'. 0101010...10101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101
